In [15]:
from pymongo import MongoClient
from datetime import datetime as dt

from src.shared import *

In [16]:
db_client = MongoClient("mongodb://localhost:27017/")
db = db_client["Lytecord"]
guilds = db["guilds"]
channels = db["channels"]
messages = db["messages"]

In [17]:
guilds.drop()
channels.drop()
messages.drop()

In [18]:
index = 0
MAX = 2 ** 22

In [19]:
for i in range(5):
    t = int(dt.now().timestamp())
    guilds.insert_one(Guild((t << 22) + index, f"Test Guild {i}").to_db_dict())
    index += 1
    index %= MAX

In [20]:
for guild_db in guilds.find():
    guild: Guild = Guild.from_db_dict(guild_db)
    for i in range(5):
        t = int(dt.now().timestamp())
        channels.insert_one(Channel((t << 22) + index, f"test channel {i}", ChannelType.TEXT, guild.id).to_db_dict())
        index += 1
        index %= MAX
channels.create_index("guild_id")

'guild_id_1'

In [21]:
for channel_db in channels.find():
    channel: Channel = Channel.from_db_dict(channel_db)
    buffer: list[dict] = []
    for i in range(150):
        t = int(dt.now().timestamp())
        buffer.insert(0, Message((t << 22) + index, channel.id, f"test message {i} in channel {channel.name} in guild {channel.guild_id}", attachment_id=0, author_id=1, timestamp=t).to_db_dict())
        index += 1
        index %= MAX
    messages.insert_many(buffer)
messages.create_index("channel_id")

'channel_id_1'

In [23]:
res = messages.find({"channel_id": 7190307204501188}).sort("_id", -1).limit(10)
[Message.from_db_dict(m) for m in res]

[Message(id=7190341555847347, channel_id=7190341555847173, content='test message 149 in channel test channel 0 in guild 7190341555847168', attachment_id=0, author_id=1, timestamp=1714311017),
 Message(id=7190341555847346, channel_id=7190341555847173, content='test message 148 in channel test channel 0 in guild 7190341555847168', attachment_id=0, author_id=1, timestamp=1714311017),
 Message(id=7190341555847345, channel_id=7190341555847173, content='test message 147 in channel test channel 0 in guild 7190341555847168', attachment_id=0, author_id=1, timestamp=1714311017),
 Message(id=7190341555847344, channel_id=7190341555847173, content='test message 146 in channel test channel 0 in guild 7190341555847168', attachment_id=0, author_id=1, timestamp=1714311017),
 Message(id=7190341555847343, channel_id=7190341555847173, content='test message 145 in channel test channel 0 in guild 7190341555847168', attachment_id=0, author_id=1, timestamp=1714311017),
 Message(id=7190341555847342, channel_id